In [3]:
import mysql.connector
from mysql.connector import Error
import tkinter as tk
from tkinter import messagebox
import tkinter.font as tkFont

# Database Configuration
HOST = "localhost"
USER = "root"
PASSWORD = "Yachna@123"  # Update with your actual password
DATABASE = "appointments_db"

# Database Connection
def create_connection():
    try:
        connection = mysql.connector.connect(
            host=HOST,
            user=USER,
            password=PASSWORD,
            database=DATABASE
        )
        if connection.is_connected():
            print("Connected to MySQL database")
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Initialize Database Tables and Add Predefined Services
def initialize_database(conn):
    cursor = conn.cursor()
    
    # Create Users table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Users (
            user_id INT AUTO_INCREMENT PRIMARY KEY,
            username VARCHAR(255) UNIQUE NOT NULL,
            password VARCHAR(255) NOT NULL
        )
    """)
    
    # Create Appointments table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Appointments (
            appointment_id INT AUTO_INCREMENT PRIMARY KEY,
            user_id INT NOT NULL,
            service VARCHAR(255) NOT NULL,
            date DATE NOT NULL,
            time TIME NOT NULL,
            status VARCHAR(50) DEFAULT 'Scheduled',
            FOREIGN KEY (user_id) REFERENCES Users(user_id)
        )
    """)
    
    # Create Services table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Services (
            service_id INT AUTO_INCREMENT PRIMARY KEY,
            service_name VARCHAR(255) NOT NULL,
            duration INT NOT NULL
        )
    """)
    
    # Clear all existing services from the Services table
    cursor.execute("DELETE FROM Services")
    conn.commit()
    
    # Predefined clinical services to be added
    clinical_services = [
        ("General Consultation", 30),
        ("Dental Checkup", 45),
        ("Blood Test", 15),
        ("X-Ray", 20),
        ("Vaccination", 10),
        ("Eye Examination", 25),
        ("Physical Therapy", 50)
    ]
    
    # Add predefined clinical services
    cursor.executemany("""
        INSERT INTO Services (service_name, duration) 
        VALUES (%s, %s)
    """, clinical_services)
    
    conn.commit()
    cursor.close()

# Main Execution
if __name__ == "__main__":
    conn = create_connection()
    if conn:
        initialize_database(conn)
        print("Database initialized with predefined services.")
        conn.close()

# Fetch Services for Dropdown
def get_services(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT service_name FROM Services")
    services = [row[0] for row in cursor.fetchall()]
    cursor.close()
    return services

# Backend Functions for User Registration, Login, and Appointment Booking
def register_user(conn, username, password):
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO Users (username, password) VALUES (%s, %s)", (username, password))
        conn.commit()
        message = "Registration successful"
    except mysql.connector.IntegrityError:
        message = "Username already taken"
    cursor.close()
    return message

def login_user(conn, username, password):
    cursor = conn.cursor()
    cursor.execute("SELECT user_id FROM Users WHERE username=%s AND password=%s", (username, password))
    user = cursor.fetchone()
    cursor.close()
    return user[0] if user else None

def book_appointment(conn, user_id, service, date, time):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT * FROM Appointments 
        WHERE user_id=%s AND date=%s AND time=%s AND status='Scheduled'
    """, (user_id, date, time))
    conflict = cursor.fetchone()
    
    if conflict:
        message = "Appointment slot already taken"
    else:
        cursor.execute("""
            INSERT INTO Appointments (user_id, service, date, time) 
            VALUES (%s, %s, %s, %s)
        """, (user_id, service, date, time))
        conn.commit()
        message = "Appointment booked successfully"
    
    cursor.close()
    return message

def view_appointments(conn, user_id):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT service, date, time, status FROM Appointments 
        WHERE user_id=%s AND status='Scheduled' 
        ORDER BY date, time
    """, (user_id,))
    appointments = cursor.fetchall()
    cursor.close()
    return appointments

# Tkinter GUI for Appointment Scheduler
class AppointmentScheduler(tk.Tk):
    def __init__(self, conn):
        super().__init__()
        self.conn = conn
        self.user_id = None
        self.title("Clinical Appointment Scheduler")
        self.geometry("800x600")
        self.config(bg="skyblue")

        # Fetch services for the dropdown menu
        self.services = get_services(self.conn)
        
        # Login Frame
        self.login_frame = tk.Frame(self, bg="pink", padx=250, pady=250)
        tk.Label(self.login_frame, text="Username", bg="black", fg="white").grid(row=0, column=0)
        tk.Label(self.login_frame, text="Password", bg="black", fg="white").grid(row=1, column=0)
        
        self.username_entry = tk.Entry(self.login_frame, width=30)
        self.password_entry = tk.Entry(self.login_frame, show="*", width=30)
        self.username_entry.grid(row=0, column=1, padx=5, pady=5)
        self.password_entry.grid(row=1, column=1, padx=5, pady=5)
        
        tk.Button(self.login_frame, text="Login", command=self.login).grid(row=2, column=0, columnspan=2)
        tk.Button(self.login_frame, text="Register", command=self.register).grid(row=3, column=0, columnspan=2)
        self.login_frame.pack()
        
        # Appointment Frame
        self.appointment_frame = tk.Frame(self, bg="pink", padx=250, pady=250)
        tk.Label(self.appointment_frame, text="Service", bg="black", fg="white").grid(row=0, column=0)
        tk.Label(self.appointment_frame, text="Date (YYYY-MM-DD)", bg="black", fg="white").grid(row=1, column=0)
        tk.Label(self.appointment_frame, text="Time (HH:MM)", bg="black", fg="white").grid(row=2, column=0)
        
        # Service Dropdown Menu
        self.service_var = tk.StringVar(self.appointment_frame)
        self.service_var.set(self.services[0])  # Set the default value to the first service
        self.service_menu = tk.OptionMenu(self.appointment_frame, self.service_var, *self.services)
        self.service_menu.grid(row=0, column=1)
        
        # Date and Time Entry
        self.date_entry = tk.Entry(self.appointment_frame)
        self.time_entry = tk.Entry(self.appointment_frame)
        self.date_entry.grid(row=1, column=1)
        self.time_entry.grid(row=2, column=1)
        
        tk.Button(self.appointment_frame, text="Book Appointment", command=self.book_appointment).grid(row=3, column=0, columnspan=2)
        tk.Button(self.appointment_frame, text="View Appointments", command=self.view_appointments).grid(row=4, column=0, columnspan=2)

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        user_id = login_user(self.conn, username, password)
        if user_id:
            self.user_id = user_id
            self.login_frame.pack_forget()
            self.appointment_frame.pack()
            messagebox.showinfo("Login", "Login successful")
        else:
            messagebox.showerror("Login", "Invalid username or password")

    def register(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        result = register_user(self.conn, username, password)
        messagebox.showinfo("Register", result)

    def book_appointment(self):
        service = self.service_var.get()
        date = self.date_entry.get()
        time = self.time_entry.get()
        result = book_appointment(self.conn, self.user_id, service, date, time)
        messagebox.showinfo("Book Appointment", result)

    def view_appointments(self):
        appointments = view_appointments(self.conn, self.user_id)
        if appointments:
            msg = "\n".join([f"{service} on {date} at {time}" for service, date, time, status in appointments])
        else:
            msg = "No scheduled appointments"
        messagebox.showinfo("Your Appointments", msg)

# Launch the Application
if __name__ == "__main__":
    conn = create_connection()
    if conn:
        app = AppointmentScheduler(conn)
        app.mainloop()
        conn.close()


Connected to MySQL database
Database initialized with predefined services.
Connected to MySQL database


In [4]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.
